In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
from scipy import stats

warnings.filterwarnings('ignore')


In [5]:
quality_data = pd.read_csv('./data/품질전처리후데이터.csv', encoding='utf-8-sig').drop("Unnamed: 0",axis=1)
asos_df = pd.read_csv('./data/asos.csv', encoding='cp949')

In [9]:
# 1) 지점, 지점명 제거
asos_df = asos_df.drop(columns=['지점', '지점명'])

# 2) 컬럼명 영어로 변경
asos_df = asos_df.rename(columns={
    '일시': 'datetime',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '습도(%)': 'humidity'
})

# 3) 강수량 결측치 → 0
asos_df['rainfall'] = asos_df['rainfall'].fillna(0)

# (선택) 4) datetime을 진짜 날짜형으로 변환
asos_df['datetime'] = pd.to_datetime(asos_df['datetime'])

# 결과 확인
print(asos_df.head())

             datetime  temperature  rainfall  humidity
0 2022-01-03 01:00:00         -4.9       0.0      76.0
1 2022-01-03 02:00:00         -5.6       0.0      78.0
2 2022-01-03 03:00:00         -6.4       0.0      80.0
3 2022-01-03 04:00:00         -6.8       0.0      83.0
4 2022-01-03 05:00:00         -7.2       0.0      85.0


In [10]:
quality_data['TAG_MIN'] = pd.to_datetime(quality_data['TAG_MIN'])
quality_data['datetime_hour'] = quality_data['TAG_MIN'].dt.floor('H')





# ASOS도 시간 단위로
asos_df['datetime_hour'] = asos_df['datetime'].dt.floor('H')

# 3) 필요 컬럼만 남기고 merge
asos_for_merge = asos_df[['datetime_hour', 'temperature', 'rainfall', 'humidity']]

merged_df = pd.merge(
    quality_data,
    asos_for_merge,
    on='datetime_hour',
    how='left'   # 공정 데이터 기준으로 붙이기
)

# 4) 필요 없으면 helper 컬럼 제거
quality_data = merged_df.drop(columns=['datetime_hour'])

print(quality_data.head())

              TAG_MIN    배정번호  건조 1존 OP  건조 2존 OP  건조로 온도 1 Zone  \
0 2022-01-03 11:22:07  102410   75.6648   30.0155            NaN   
1 2022-01-03 11:22:08  102410   75.6706   32.2732            NaN   
2 2022-01-03 11:22:09  102410   75.6776   32.1592        98.8533   
3 2022-01-03 11:22:11  102410   75.8656   30.8312        98.7918   
4 2022-01-03 11:22:12  102410   73.6468   29.5274        98.7918   

   건조로 온도 2 Zone      세정기   소입1존 OP  소입2존 OP   소입3존 OP  ...  소입로 온도 2 Zone  \
0            NaN  68.4386  72.84030  59.7862  51.71690  ...        859.854   
1            NaN  68.4386  78.44150  61.6286  50.44530  ...        859.780   
2       99.14600  68.4386  78.10990  61.5414  52.01960  ...        859.780   
3       99.17675  68.4999  77.50725  60.6663  52.69425  ...        859.842   
4       99.20750  68.4386  76.02620  61.1634  51.69150  ...        859.791   

   소입로 온도 3 Zone  소입로 온도 4 Zone  솔트 컨베이어 온도 1 Zone  솔트 컨베이어 온도 2 Zone  \
0            NaN            NaN                Na

In [11]:

label_data = pd.read_csv('./data/label.csv', encoding='utf-8-sig')

defect_rates = []
for _, row in label_data.iterrows():
    assignment_num = row['배정번호']
    good_qty = row['양품수량']
    defect_qty = row['불량수량']
    total_qty = row['총수량']
    
    if total_qty > 0:
        defect_rate = (defect_qty / total_qty) * 100
    else:
        defect_rate = 0
        
    defect_rates.append({
        '배정번호': assignment_num,
        '불량률(%)': defect_rate
    })

defect_df = pd.DataFrame(defect_rates)

process_columns = [col for col in quality_data.columns if col not in ['TAG_MIN', '배정번호']]

min_max_data = []
for assignment_num in quality_data['배정번호'].unique():
    assignment_data = quality_data[quality_data['배정번호'] == assignment_num]
    
    row_data = {'배정번호': assignment_num}
    
    for col in process_columns:
        if assignment_data[col].notna().any():
            col_data = assignment_data[col].dropna()
            row_data[f'{col}_최고'] = col_data.max()
            row_data[f'{col}_최저'] = col_data.min()
        else:
            row_data[f'{col}_최고'] = np.nan
            row_data[f'{col}_최저'] = np.nan
    
    min_max_data.append(row_data)

min_max_df = pd.DataFrame(min_max_data)

unified_df = pd.merge(defect_df, min_max_df, on='배정번호', how='inner')

unified_df.to_csv('./data/unified.csv', index=False, encoding='utf-8-sig')

In [13]:
def compare_temperature_variables(low_group, high_group):
    temp_cols = [col for col in low_group.columns if col not in ['배정번호', '불량률(%)']]
    results = []

    for col in temp_cols:
        low_values = low_group[col].dropna()
        high_values = high_group[col].dropna()

        if len(low_values) == 0 or len(high_values) == 0:
            continue

        low_mean = low_values.mean()
        low_std = low_values.std()
        low_median = low_values.median()
        low_min = low_values.min()
        low_max = low_values.max()

        high_mean = high_values.mean()
        high_std = high_values.std()
        high_median = high_values.median()
        high_min = high_values.min()
        high_max = high_values.max()

        mean_diff = high_mean - low_mean
        mean_diff_pct = (mean_diff / low_mean * 100) if low_mean != 0 else 0

        if len(low_values) >= 2 and len(high_values) >= 2:
            try:
                t_stat, t_pval = stats.ttest_ind(high_values, low_values)
            except:
                t_stat, t_pval = np.nan, np.nan
        else:
            t_stat, t_pval = np.nan, np.nan

        if len(low_values) >= 1 and len(high_values) >= 1:
            try:
                u_stat, u_pval = stats.mannwhitneyu(high_values, low_values, alternative='two-sided')
            except:
                u_stat, u_pval = np.nan, np.nan
        else:
            u_stat, u_pval = np.nan, np.nan

        pooled_std = np.sqrt((low_std**2 + high_std**2) / 2)
        cohens_d = mean_diff / pooled_std if pooled_std != 0 else 0

        results.append({
            '온도 변수': col,
            '불량률0_평균': low_mean,
            '불량률0_표준편차': low_std,
            '불량률0_중앙값': low_median,
            '불량률0_최소': low_min,
            '불량률0_최대': low_max,
            '상위10_평균': high_mean,
            '상위10_표준편차': high_std,
            '상위10_중앙값': high_median,
            '상위10_최소': high_min,
            '상위10_최대': high_max,
            '평균_차이': mean_diff,
            '평균_차이(%)': mean_diff_pct,
            't_통계량': t_stat,
            't_p-value': t_pval,
            'U_통계량': u_stat,
            'U_p-value': u_pval,
            'Cohens_d': cohens_d,
        })

    results_df = pd.DataFrame(results)
    results_df['절댓값_차이'] = results_df['평균_차이'].abs()
    results_df = results_df.sort_values('절댓값_차이', ascending=False)
    results_df = results_df.drop('절댓값_차이', axis=1)

    return results_df

In [15]:
def analyze_temperature_range(zero_defect_data, variable_name):
    if variable_name not in zero_defect_data.columns:
        return None

    values = zero_defect_data[variable_name].dropna()

    if len(values) == 0:
        return None

    result = {
        '변수명': variable_name,
        '샘플수': len(values),
        '최소값': values.min(),
        '최대값': values.max(),
        '평균': values.mean(),
        '중앙값': values.median(),
        '표준편차': values.std(),
        '범위': values.max() - values.min(),
        '값_목록': list(values.values),
    }

    return result

In [16]:
# 통합 CSV 생성 - 배정번호 별 불량률과 공정 별 최소/최대 온도 추출, 저장


# 데이터 호출
df = pd.read_csv('./data/unified.csv', encoding='utf-8-sig')

# 불량률이 정확히 0.0%인 배치만 선정
zero_defect_group = df[df['불량률(%)'] == 0.0].copy()

# 불량률 상위 10개인 배치 선정
df_sorted = df.sort_values('불량률(%)', ascending=False)
high_defect_group = df_sorted.head(10).copy()

# 비교 수행 Cohen's d와 P-Value Test를 통해 유의미한 차이가 있는 공정 식별
comparison_df = compare_temperature_variables(zero_defect_group, high_defect_group)

# Cohen's d의 절대값기준 정렬 후 상위 3개 항목 추출
comparison_df['Cohens_d_abs'] = comparison_df['Cohens_d'].abs()
df_sorted = comparison_df.sort_values('Cohens_d_abs', ascending=False)
top3 = df_sorted.head(3)

# 해당 항목들에 대해서 공정 값 범위 추출
results = []
for var_name in top3['온도 변수'].values:
    result = analyze_temperature_range(zero_defect_group, var_name)
    if result:
        results.append(result)
results


[{'변수명': '소입로 온도 1 Zone_최저',
  '샘플수': 3,
  '최소값': 850.135,
  '최대값': 855.002,
  '평균': 852.0326666666666,
  '중앙값': 850.961,
  '표준편차': 2.604471987434919,
  '범위': 4.866999999999962,
  '값_목록': [855.002, 850.135, 850.961]},
 {'변수명': '건조로 온도 2 Zone_최저',
  '샘플수': 3,
  '최소값': 98.8823,
  '최대값': 99.4547,
  '평균': 99.07339999999999,
  '중앙값': 98.8832,
  '표준편차': 0.33021579308082827,
  '범위': 0.5724000000000018,
  '값_목록': [99.4547, 98.8832, 98.8823]},
 {'변수명': '소입로 온도 4 Zone_최저',
  '샘플수': 3,
  '최소값': 858.267,
  '최대값': 858.713,
  '평균': 858.4326666666666,
  '중앙값': 858.318,
  '표준편차': 0.2441113134070537,
  '범위': 0.4459999999999127,
  '값_목록': [858.318, 858.267, 858.713]}]

#### 불량률 = 0% 그룹 (3개)
| 순위 | 배정번호 | 불량률(%) |
|------|---------|----------|
| 1 | 104126 | 0.0000% |
| 2 | 135615 | 0.0000% |
| 3 | 148069 | 0.0000% |
- **완벽한 품질**: 불량품 0개

#### 불량률 상위 10개 그룹
| 순위 | 배정번호 | 불량률(%) |
|------|---------|----------|
| 1 | 128795 | 0.3685% |
| 2 | 116413 | 0.2480% |
| 3 | 128800 | 0.2214% |
| 4 | 140920 | 0.1990% |
| 5 | 141145 | 0.1727% |
| 6 | 126806 | 0.1092% |
| 7 | 138774 | 0.1033% |
| 8 | 140548 | 0.0895% |
| 9 | 141554 | 0.0870% |
| 10 | 125919 | 0.0804% |
- **평균 불량률**: 0.1679%
- **불량률 범위**: 0.0804% ~ 0.3685%

### 뷸량률 0% 그룹과 상위 10개 그룹에서 유의미하게 차이나는 공정
| 순위 | 온도 변수 | 불량률0<br>평균 | 상위10<br>평균 | 평균<br>차이 | 차이(%) | Cohen's d | 효과크기 |
|------|-----------|-----------------|----------------|--------------|---------|-----------|----------|
| 1 | 소입로 온도 1 Zone_최저 | 852.03 | 845.87 | -6.16 | -0.72% | -2.305 | 큼 |
| 2 | 건조로 온도 2 Zone_최저 | 99.07 | 98.34 | -0.73 | -0.74% | -2.149 | 큼 |
| 3 | 소입로 온도 4 Zone_최저 | 858.43 | 858.84 | +0.41 | +0.05% | +1.680 | 큼 |

<br>

### 해당 공정 통계값
| 순위 | 온도 변수 | 최소값 | 최대값 | 평균 | 중앙값 | 표준편차 | 범위 |
|------|-----------|---------|--------|------|--------|----------|------|
| 1 | 소입로 온도 1 Zone_최저 | 850.13 | 855.00 | 852.03 | 850.96 | 2.60 | 4.87 |
| 2 | 건조로 온도 2 Zone_최저 | 98.88 | 99.45 | 99.07 | 98.88 | 0.33 | 0.57 |
| 3 | 소입로 온도 4 Zone_최저 | 858.27 | 858.71 | 858.43 | 858.32 | 0.24 | 0.45 |
